In [1]:
import sys

sys.path.append("..")
from typing import List

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from pydantic import BaseModel, Field

from brain.sdk import BrainClient

In [2]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
from typing import List, Union

from pydantic import BaseModel, Field


class Tool(BaseModel):
    script: str = Field(..., description="The Python script to be executed. Ensure it handles errors and outputs structured results.")
    pip_install: List[str] = Field(..., description="The list of pip packages required to execute the script.")
    completed: Union[bool, str] = Field(..., description="Indicates if the task is successfully completed. True if completed, otherwise a string explaining why the script cannot be generated.")

In [4]:
@brain_client.reasoner(schema=Tool)
def autonomous_tool(task: str):
    """
    Generates standalone, executable Python scripts with no external dependencies.
    """
    system_prompt = """You are a Python script generator focused on creating fully autonomous code. Your scripts must be completely self-contained:

DO NOT generate scripts that:
- Require external inputs (API keys, file paths, user prompts, etc.)
- Use mock/placeholder data
- Need manual configuration
- Access local files or environment variables
- Connect to external services without built-in test data
- Depend on system-specific resources
- Use unreliable or niche packages

Instead:
- Use standard library when possible
- Include test data within the script
- Handle all edge cases and errors
- Use only widely-used PyPI packages if needed
- Generate any required data programmatically
- If we cannot do it without external dependencies, explain why in 'completed' field and guide the tool maker to say it cannot be done

If task requires unavoidable external dependencies, explain why in 'completed' field."""

    user_prompt = f"""Generate a standalone Python script for: "{task}"

Return:
- script: Fully executable Python code
- pip_install: Required packages (empty list if none needed)
- completed: True if successful, or explanation why task cannot be autonomous"""

    return system_prompt, user_prompt
autonomous_tool_id=autonomous_tool.register()

In [5]:
import os
import shutil
import subprocess
import tempfile
import venv


def execute_script_with_temp_venv(script: str, pip_install: list) -> str:
    """
    Creates a temporary virtual environment, installs required pip packages, 
    executes the provided script, and returns the formatted output.

    Args:
        script (str): The Python script to execute.
        pip_install (list): List of pip packages to install in the virtual environment.

    Returns:
        str: Formatted string containing stdout, stderr, and status of execution.
    """
    result_output = f"Script:\n{script}\n\nRequirements:\n{pip_install}\n\n"
    temp_dir = None

    try:
        # Step 1: Create a temporary directory for the virtual environment
        temp_dir = tempfile.mkdtemp()
        venv_dir = os.path.join(temp_dir, "venv")
        venv.create(venv_dir, with_pip=True)

        # Step 2: Paths for the virtual environment
        venv_python = os.path.join(venv_dir, "bin", "python") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "python")
        venv_pip = os.path.join(venv_dir, "bin", "pip") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "pip")

        # Step 3: Install required packages
        if pip_install:
            install_command = [venv_pip, "install"] + pip_install
            subprocess.run(install_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Step 4: Write the script to a temporary file
        script_path = os.path.join(temp_dir, "script.py")
        with open(script_path, "w") as script_file:
            script_file.write(script)

        # Step 5: Execute the script
        execute_command = [venv_python, script_path]
        process = subprocess.run(
            execute_command, check=False, capture_output=True, text=True
        )

        # Step 6: Format output
        if process.returncode == 0:
            result_output += f"Execution Status: SUCCESS\n\nSTDOUT:\n{process.stdout.strip()}\n"
        else:
            result_output += (
                f"Execution Status: FAILURE\n\nSTDERR:\n{process.stderr.strip()}\n"
                f"STDOUT (if any):\n{process.stdout.strip()}\n"
            )

    except subprocess.CalledProcessError as e:
        # Handle pip installation or script execution errors
        result_output += (
            f"Execution Status: ERROR\n\nERROR:\n{str(e)}\n"
            f"Command Output:\n{e.output if e.output else 'No Output'}\n"
        )
    except Exception as e:
        # Catch any unexpected exceptions
        result_output += f"Execution Status: ERROR\n\nUnexpected Error: {str(e)}\n"
    finally:
        # Step 7: Cleanup temporary directory
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

    return result_output

In [6]:
from typing import Optional, Union

from pydantic import BaseModel, Field


class Success(BaseModel):
    final_answer: str = Field(..., description="The final result or answer to the task if the execution succeeded.")

class Failure(BaseModel):
    analysis: str = Field(..., description="A detailed explanation of why the tool execution failed.")
    suggestions: str = Field(..., description="Suggestions for fixing the issue or alternative approaches to achieve the task.")

class Result(BaseModel):
    success: bool = Field(..., description="Indicates if the tool execution was successful.")
    result: Union[Success, Failure] = Field(..., description="The result of the analysis, including success or failure details.")
    
@brain_client.reasoner(schema=Result)
def result_analyzer(tool_output: str, task: str):
    """
    Analyzes script execution results focusing on zero human intervention requirement.
    """
    system_prompt = """Analyze Python script results for true autonomy - script must run without any human intervention.

Success case:
- Mention success as True
Extract meaningful result from tool_output and present as:
- Direct answer addressing the original task
- Natural, human-readable format
- No technical details unless relevant
- No mention of script/code aspects


Failure case - Check for:
1. Manual Requirements (like these, but it can be more):
   - API keys/credentials needed
   - File paths/names input
   - User prompts/inputs
   - Configuration steps
   - Runtime user decisions

2. Script should:
   - Use standard library or pip packages
   - Make API calls if no credentials needed
   - Handle all data programmatically
   - Run start-to-finish autonomously

Key principle:
If task requires ANY manual steps (API keys, inputs, etc):
- Clearly state why it can't be autonomous
- Don't suggest mock/placeholder solutions
- Only suggest truly automatic alternatives"""

    user_prompt = f"""Analyze result for: "{task}"
Output: "{tool_output}"

Success: Direct task response
Failure: Explain manual intervention needs"""

    return system_prompt, user_prompt
result_analyzer_id = result_analyzer.register()

In [ ]:
@brain_client.multi_agent
def autonomous_tool(task: str,max_trials=5,verbose=True):
    for i in range(max_trials):
        print(f"{'='*10} Iteration {i+1} {'='*10}")
        if i!=0:
            if not res.success:
                task_context=(f"Last tool failed becasue {res.result}. Last tool script was {tool_output.script} and pip install was {tool_output.pip_install}"
                            "use this information to generate the next tool for the task\n")
        else:
            task_context=""
        tool_output = brain_client.use(autonomous_tool_id)(task=task_context+task)
        exec_output=execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)
        res=brain_client.use(result_analyzer_id)(tool_output=exec_output, task=task)
        if verbose:
            print(res.result)
        if res.success:
            if verbose:
                print(f"Tool generated:\n {exec_output}")
            break
    return res.result
    

In [8]:
task="search for top 3 articles on autonomous driving and give back the abstracts"
autonomous_tool(task,max_trials=5,verbose=True)

========== Iteration 1 ==========
final_answer='The script successfully searched for articles on autonomous driving and retrieved their abstracts. It efficiently found three relevant articles, extracting their titles and abstracts without any need for human intervention.'
Tool generated:
 import requests
from bs4 import BeautifulSoup

# Function to search for articles about autonomous driving

def search_articles(query, num_articles=3):
    url = f'https://www.google.com/search?q={query}&num={num_articles}'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
    except requests.exceptions.RequestException as e:
        return {'error': f'HTTP Request failed: {e}'}

    # Parse the response using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []
    for item in soup.find_all('div', class_='BVG0Nb'):  # Adjustclass according to Google results
        title = item.find('h3')  # Article t

Success(final_answer='The script successfully searched for articles on autonomous driving and retrieved their abstracts. It efficiently found three relevant articles, extracting their titles and abstracts without any need for human intervention.')

In [9]:
task="calculate the sum of numbers from 1 to 100_0000_000"
autonomous_tool(task,max_trials=5,verbose=True)

========== Iteration 1 ==========
final_answer='The sum of numbers from 1 to 1,000,000 is: 500,000,500,000.'
Tool generated:
 def calculate_sum(start, end):
    """
    Calculate the sum of numbers from start to end (inclusive).
    Handles large numbers and edge cases.
    """
    if start < 1 or end < 1:
        raise ValueError('Start and end values must be positive integers.')
    if start > end:
        raise ValueError('Start value must be less than or equal to end value.')
    return sum(range(start, end + 1))

if __name__ == '__main__':
    try:
        start_number = 1
        end_number = 1000000
        total_sum = calculate_sum(start_number, end_number)
        print(f'The sum of numbers from {start_number} to {end_number} is: {total_sum}')
    except Exception as e:
        print(f'An error occurred: {e}')


Success(final_answer='The sum of numbers from 1 to 1,000,000 is: 500,000,500,000.')